In [2]:
import sqlite3 

In [3]:
conn = sqlite3.connect('neurons.db') # opens database
c = conn.cursor()

In [4]:
test = "test"
c.execute(f"""

CREATE TABLE {test} (
    cell_name TEXT,
    mouse_name TEXT,
    avg_dff_trace REAL
)

""")
conn.commit()

In [21]:
c.execute("INSERT INTO neurons VALUES ('C01','BLA-Insc-1','20.5')")
conn.commit()

In [11]:
cells = [
    ('C02','BLA-Insc-3','20.5'),
    ('C03','BLA-Insc-3','20.5'),
    ('C04','BLA-Insc-3','20.5')
]

c.executemany("INSERT INTO neurons VALUES (?,?,?)", cells)
conn.commit()

In [3]:
c.execute("SELECT * FROM neurons")
rows = c.fetchall()
for row in rows:
    print(row)

('C01', 'BLA-Insc-1', 20.5)
('C02', 'BLA-Insc-1', 20.5)
('C03', 'BLA-Insc-1', 20.5)
('C04', 'BLA-Insc-1', 20.5)
('C02', 'BLA-Insc-3', 20.5)
('C03', 'BLA-Insc-3', 20.5)
('C04', 'BLA-Insc-3', 20.5)


In [8]:
colnames = []
for description in c.description :
    colnames.append(description[0])

print(colnames)

['cell_name', 'mouse_name', 'avg_dff_trace']


In [7]:
# we have row ids
# most database systems you have to add in the primary key and have an incrementing mechanism
# with, sqlite you don't have to do that
c.execute("SELECT rowid, * FROM neurons")
rows = c.fetchall()
for row in rows:
    print(row)

(1, 'C01', 'BLA-Insc-1', 20.5)
(2, 'C02', 'BLA-Insc-1', 20.5)
(3, 'C03', 'BLA-Insc-1', 20.5)
(4, 'C04', 'BLA-Insc-1', 20.5)
(5, 'C02', 'BLA-Insc-3', 20.5)
(6, 'C03', 'BLA-Insc-3', 20.5)
(7, 'C04', 'BLA-Insc-3', 20.5)


In [6]:
#Searching if a row exists already
#Does C01 already exists?

for row in c.execute("""SELECT * FROM neurons GROUP BY cell_name HAVING cell_name = 'C02' """):
    print(row)

('C02', 'BLA-Insc-1', 20.5)


In [24]:
# THIS IS IT!: for when you want to look up if a cell already exists!
result = None
look_for = "C01"
for row in c.execute("""SELECT * FROM neurons GROUP BY cell_name HAVING cell_name = (?)""", (look_for,)):
    result = row

print(f"Result: {result}")
if isinstance(result, tuple):
        print("Cell exists already!")
else:
    print("Cell does not exist!")


Result: ('C01', 'BLA-Insc-1', 20.5)
Cell exists already!


In [26]:
result = None
look_for = "C07"
for row in c.execute("""SELECT * FROM neurons WHERE neurons.cell_name = (?)""", (look_for,)):
    result = row

print(f"Result: {result}")
if isinstance(result, tuple):
        print("Cell exists already!")
else:
    print("Cell does not exist!")

Result: None
Cell does not exist!


In [15]:
#THIS IS IT FOR GETTING ALL TUPLES YOU NEED

for row in c.execute("""SELECT * FROM neurons WHERE neurons.cell_name='C02' """):
    print(row)
    print(type(row))

('C02', 'BLA-Insc-1', 20.5)
<class 'tuple'>
('C02', 'BLA-Insc-3', 20.5)
<class 'tuple'>


In [28]:
#GETTING A SPEFIC VALUE
result = None
look_for = "C01"
for row in c.execute("""SELECT avg_dff_trace FROM neurons WHERE neurons.cell_name = (?)""", (look_for,)):
    result = row

print(f"Result: {result}")
print(type(result))
# so will always return a tuple, have a function that converts tuple of single element to that element

Result: (20.5,)
<class 'tuple'>


In [29]:
class NeuronSessionTestManager:
    def __init__(self, name):
        self.name = name
        self.d = {"test_1" : "+"}

c.execute("""

CREATE TABLE neurons2 (
    cell_name TEXT,
    test_manager BLOB
)

""")
conn.commit()

In [33]:
cell = "C01"
manager_1 = NeuronSessionTestManager(cell)
c.execute("INSERT INTO neurons2 VALUES (?,?)", (cell, manager_1,))
conn.commit()

InterfaceError: Error binding parameter 1 - probably unsupported type.

In [12]:
conn.close()